In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interpolate

from DarkCapPy import *
import DarkCapPy.DarkPhoton as DP



In [ ]:
mx = 1000
ma = 1
epsilon = 1e-8
alpha = 1/137
alphax = 0.035
tauCross = DP.tauCross

#cap1 = DP.cCap(mx, ma, epsilon, alpha, alphax)
kappa0 = DP.kappa_0(mx, alpha)
print(kappa0)
cap2 = DP.cCapQuick(mx, ma, epsilon, alphax, kappa0)
print(cap2)


sommerfeld = DP.thermAvgSommerfeld(mx, ma, alphax)
sigma = DP.sigmaVtree(mx, ma, alphax)
ann = DP.cAnn(mx, sigma, sommerfeld)

#tau = DP.tau(cap1, ann)
tau = DP.tau(cap2, ann)

#gammaAnn = DP.gammaAnn(cap1, ann)
gammaAnn = DP.gammaAnn(cap2, ann)


L = DP.decayLength(mx, ma, epsilon, 1)
Edecay = DP.epsilonDecay(L)

signal = DP.iceCubeSignal(gammaAnn, Edecay, DP.yr2s(10))

In [ ]:
#### TEST

mx = 1000
ma = 1
epsilon = 1e-8
#alpha = 0
alpha = 0.001
alphax = 0.035
tauCross = DP.tauCross

#cap1 = DP.cCap(mx, ma, epsilon, alpha, alphax)
kappa0 = DP.kappa_0(mx, alpha)
print(kappa0)
cap2 = DP.cCapQuick(mx, ma, epsilon, alphax, kappa0)
print(cap2)


sommerfeld = DP.thermAvgSommerfeld(mx, ma, alphax)
sigma = DP.sigmaVtree(mx, ma, alphax)
ann = DP.cAnn(mx, sigma, sommerfeld)

#tau = DP.tau(cap1, ann)
tau = DP.tau(cap2, ann)

#gammaAnn = DP.gammaAnn(cap1, ann)
gammaAnn = DP.gammaAnn(cap2, ann)


L = DP.decayLength(mx, ma, epsilon, 1)
Edecay = DP.epsilonDecay(L)

signal = DP.iceCubeSignal(gammaAnn, Edecay, DP.yr2s(10))